# SFC, FC & SC - LOBE vs HCP #

## Importing packages and setting paths ##

In [239]:
import pandas as pd
import nibabel as nib
from glob import glob
import numpy as np
import statsmodels.api as sp
#imported the full paths since it wasn't working otherwise - not sure why 
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.multitest import fdrcorrection


In [240]:
#first load up the label dseg table
atlas='schaefer'

df_labels = pd.read_csv(f'resources/atlas/atlas-{atlas}_dseg.tsv',sep='\t')
df_labels

df_network_names = df_labels.networks.unique()
df_network_names

df_tabular = pd.read_csv('resources/merged_tabular.tsv',sep='\t')
df_tabular



,participant_id,dataset,age,sex,age_onset,ep_duration,pnes,focal_seizure_type,generalized_seizures,dre,...,loc_eeg_parietal,loc_eeg_temporal,loc_mri_frontal,loc_mri_occipital,loc_mri_parietal,loc_mri_temporal,loc_pet_frontal,loc_pet_occipital,loc_pet_parietal,loc_pet_temporal
0,sub-HCD0001305,HCP,11.916667,M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-HCD0021614,HCP,9.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub-HCD0026119,HCP,15.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub-HCD0041822,HCP,17.416667,M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-HCD0042420,HCP,18.333333,M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,sub-029,LOBE,16.000000,M,15.0,1.0,0.0,motor,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,sub-031,LOBE,9.000000,M,4.0,5.0,0.0,motor,0.0,NaN,...,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
369,sub-033,LOBE,7.000000,M,4.0,3.0,0.0,non-motor,1.0,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,sub-035,LOBE,7.000000,M,2.0,5.0,0.0,motor,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [241]:
fc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_bold.pconn.nii'
sc_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-32k_atlas-schaefer_struc.pconn.nii'
sfc_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_sfc.pscalar.nii'




In [242]:
sfc_paths={}
sfc_paths['HCP'] = sorted(glob(sfc_pattern.format(dataset='HCP',subject='*')))
sfc_paths['LOBE'] = sorted(glob(sfc_pattern.format(dataset='LOBE',subject='*')))

fc_paths={}
fc_paths['HCP'] = sorted(glob(fc_pattern.format(dataset='HCP',subject='*')))
fc_paths['LOBE'] = sorted(glob(fc_pattern.format(dataset='LOBE',subject='*')))

sc_paths={}
sc_paths['HCP'] = sorted(glob(sc_pattern.format(dataset='HCP',subject='*')))
sc_paths['LOBE'] = sorted(glob(sc_pattern.format(dataset='LOBE',subject='*')))

#print(sfc_paths)
#print(fc_paths)
#print(sc_paths)

In [243]:
#from snakebids.utils.snakemake_io import glob_wildcards
#subjects = glob_wildcards(sfc_pattern.format(dataset='HCP',subject='{subject}'))


In [244]:
hacky_hcp_subjects = [ path[12:26] for path in sfc_paths['HCP']]
hacky_hcp_subjects[0]

'sub-HCD0001305'

In [245]:
sfc={}
fc={}
sc={}
num_parcels = nib.load(sfc_paths['LOBE'][0]).get_fdata().shape[1]

for dataset in datasets:
    sfc[dataset] = np.concatenate(  [ nib.load(path).get_fdata()  for path in sfc_paths[dataset]] ,axis=0)
    fc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in fc_paths[dataset]] ,axis=0)
    sc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sc_paths[dataset]] ,axis=0)

In [246]:
df_tabular

,participant_id,dataset,age,sex,age_onset,ep_duration,pnes,focal_seizure_type,generalized_seizures,dre,...,loc_eeg_parietal,loc_eeg_temporal,loc_mri_frontal,loc_mri_occipital,loc_mri_parietal,loc_mri_temporal,loc_pet_frontal,loc_pet_occipital,loc_pet_parietal,loc_pet_temporal
0,sub-HCD0001305,HCP,11.916667,M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-HCD0021614,HCP,9.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub-HCD0026119,HCP,15.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub-HCD0041822,HCP,17.416667,M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-HCD0042420,HCP,18.333333,M,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,sub-029,LOBE,16.000000,M,15.0,1.0,0.0,motor,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,sub-031,LOBE,9.000000,M,4.0,5.0,0.0,motor,0.0,NaN,...,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
369,sub-033,LOBE,7.000000,M,4.0,3.0,0.0,non-motor,1.0,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,sub-035,LOBE,7.000000,M,2.0,5.0,0.0,motor,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [247]:

#this adds RSFC features as new columns to the df_tabular

df_list=[]
for dataset in datasets:
    df_=pd.DataFrame(sfc[dataset],
                 columns=('RSFC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = list(df_tabular_['participant_id'])
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [248]:
sfc['HCP'].shape

(341, 300)

# Parcel-Wise, LOBE vs HCP #

## SFC ##

In [249]:
fc['LOBE'].shape

(23, 300, 300)

In [250]:
x1 = sfc['HCP']
x2 = sfc['LOBE']

In [251]:
x2.shape

(23, 300)

In [252]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

(300,)

In [253]:
pvals

array([2.19823553e-01, 3.51062986e-01, 1.27458705e-01, 3.14298849e-01,
       8.37194778e-01, 9.59317585e-01, 1.44900582e-01, 5.88645247e-01,
       1.81285733e-01, 8.71245333e-01, 8.46768988e-01, 5.88242812e-02,
       1.21386051e-01, 9.27871120e-01, 4.46061043e-01, 5.36327231e-01,
       7.26868159e-02, 2.81507381e-01, 9.15475581e-01, 9.78991575e-01,
       1.28328289e-01, 6.28069987e-02, 1.27124049e-02, 5.98235067e-06,
       7.62524512e-01, 6.12949289e-01, 9.42084582e-01, 4.18739773e-01,
       9.24373902e-01, 8.74420119e-01, 7.46129789e-01, 6.28451809e-01,
       2.51943482e-01, 9.05347778e-01, 8.52242528e-01, 8.76505596e-01,
       3.26909733e-01, 5.67266058e-01, 6.67390955e-01, 2.79830324e-01,
       6.76384236e-01, 9.98892177e-01, 5.31938329e-01, 3.54870452e-01,
       3.47470540e-01, 3.68240654e-01, 9.58443228e-01, 5.12309304e-01,
       1.06544358e-01, 1.09262732e-01, 7.52968726e-01, 5.56545608e-01,
       7.68358659e-01, 9.37737275e-01, 4.27854456e-01, 2.42780546e-01,
      

In [254]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [255]:
rejected

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

# Functional Connectivity #

## Average FC per parcel - group difference ##

In [256]:
#set diagonal to be nan
x1 = fc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = fc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [257]:
x1.shape

(341, 300, 300)

In [258]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [259]:
x1.shape

(341, 300)

In [260]:
x2.shape

(23, 300)

In [261]:
#add FC parcel avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('RFC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = list(df_tabular_['participant_id'])
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [262]:
print(df_tabular)

    participant_id dataset        age sex  age_onset  ep_duration  pnes  \
0   sub-HCD0001305     HCP  11.916667   M        NaN          NaN   NaN   
1   sub-HCD0021614     HCP   9.166667   F        NaN          NaN   NaN   
2   sub-HCD0026119     HCP  15.166667   F        NaN          NaN   NaN   
3   sub-HCD0041822     HCP  17.416667   M        NaN          NaN   NaN   
4   sub-HCD0042420     HCP  18.333333   M        NaN          NaN   NaN   
..             ...     ...        ...  ..        ...          ...   ...   
18         sub-029    LOBE  16.000000   M       15.0          1.0   0.0   
19         sub-031    LOBE   9.000000   M        4.0          5.0   0.0   
20         sub-033    LOBE   7.000000   M        4.0          3.0   0.0   
21         sub-035    LOBE   7.000000   M        2.0          5.0   0.0   
22         sub-038    LOBE  10.000000   F        7.0          3.0   0.0   

   focal_seizure_type  generalized_seizures  dre  ...  \
0                 NaN                   Na

### Individual T-test ###

In [263]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

(300,)

### FDR Correction ###

In [264]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [265]:
# show if null is rejected after FDR correction, True = significant
rejected

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

# Structural Connectivity #

## Average SC per parcel - group difference ##

In [266]:
#set diagonal to be nan
x1 = sc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [267]:
x1.shape

(341, 300, 300)

In [268]:
x1 = np.nanmean(x1,axis=2)
x1.shape
x2 = np.nanmean(x2,axis=2)
x2.shape

(23, 300)

In [269]:
x1.shape

(341, 300)

In [270]:
x2.shape

(23, 300)

In [271]:
#add SC parcel avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('RSC_'+df_labels.name))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = list(df_tabular_['participant_id'])
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [272]:
df_tabular

,participant_id,dataset,age,sex,age_onset,ep_duration,pnes,focal_seizure_type,generalized_seizures,dre,...,RSC_7Networks_RH_Default_PFCdPFCm_7,RSC_7Networks_RH_Default_PFCdPFCm_8,RSC_7Networks_RH_Default_PFCdPFCm_9,RSC_7Networks_RH_Default_PFCdPFCm_10,RSC_7Networks_RH_Default_pCunPCC_1,RSC_7Networks_RH_Default_pCunPCC_2,RSC_7Networks_RH_Default_pCunPCC_3,RSC_7Networks_RH_Default_pCunPCC_4,RSC_7Networks_RH_Default_pCunPCC_5,RSC_7Networks_RH_Default_pCunPCC_6
0,sub-HCD0001305,HCP,11.916667,M,NaN,NaN,NaN,NaN,NaN,NaN,...,14.844157,15.208391,20.545357,20.011699,9.874583,7.119848,8.502915,8.000757,9.652369,5.166225
1,sub-HCD0021614,HCP,9.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,14.454994,8.395796,13.738416,14.700760,7.839966,5.910502,6.833241,6.978524,8.193208,3.897537
2,sub-HCD0026119,HCP,15.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,13.889760,12.957404,18.191067,20.401691,6.084737,6.723275,7.880876,7.008210,10.270747,3.955877
3,sub-HCD0041822,HCP,17.416667,M,NaN,NaN,NaN,NaN,NaN,NaN,...,21.165592,13.562098,17.818852,22.240259,5.806375,5.222681,6.741655,6.136430,8.931968,5.828264
4,sub-HCD0042420,HCP,18.333333,M,NaN,NaN,NaN,NaN,NaN,NaN,...,22.301299,8.759729,18.507906,15.216472,5.089299,3.803380,4.383564,6.403444,5.134533,3.186134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,sub-029,LOBE,16.000000,M,15.0,1.0,0.0,motor,1.0,0.0,...,19.028710,8.342776,18.587800,16.902234,8.433928,5.127071,6.260919,4.993658,8.733412,2.258928
19,sub-031,LOBE,9.000000,M,4.0,5.0,0.0,motor,0.0,NaN,...,16.960664,16.462967,19.383860,25.676945,10.450205,6.463145,8.535226,7.709845,8.528428,3.513172
20,sub-033,LOBE,7.000000,M,4.0,3.0,0.0,non-motor,1.0,1.0,...,14.291480,10.521352,19.107467,18.687367,6.589493,5.635943,8.236162,7.513440,8.032734,2.817005
21,sub-035,LOBE,7.000000,M,2.0,5.0,0.0,motor,1.0,0.0,...,15.359479,9.876812,19.154256,25.053800,6.786074,5.924701,7.349385,7.168108,6.827848,2.840658


### Individual T-test ###

In [273]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape

(300,)

In [274]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [275]:
rejected

array([False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

# Network-Wise, LOBE vs HCP #

In [276]:
fc_network_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_netbold.pconn.nii'
sc_network_pattern='results/{dataset}/sub-{subject}/dwi/sub-{subject}_den-32k_atlas-schaefer_netstruc.pconn.nii'
sfc_network_pattern='results/{dataset}/sub-{subject}/func/sub-{subject}_task-rest_den-32k_desc-preproc_denoise-24HMP8PhysSpikeReg_fwhm-5_atlas-schaefer_netsfc.pconn.nii'


datasets=['HCP','LOBE']

In [277]:
sfc_netpaths={}
sfc_netpaths['HCP'] = sorted(glob(sfc_network_pattern.format(dataset='HCP',subject='*')))
sfc_netpaths['LOBE'] = sorted(glob(sfc_network_pattern.format(dataset='LOBE',subject='*')))

fc_netpaths={}
fc_netpaths['HCP'] = sorted(glob(fc_network_pattern.format(dataset='HCP',subject='*')))
fc_netpaths['LOBE'] = sorted(glob(fc_network_pattern.format(dataset='LOBE',subject='*')))

sc_netpaths={}
sc_netpaths['HCP'] = sorted(glob(sc_network_pattern.format(dataset='HCP',subject='*')))
sc_netpaths['LOBE'] = sorted(glob(sc_network_pattern.format(dataset='LOBE',subject='*')))

#print(sfc_netpaths)
#print(fc_netpaths)
#print(sc_netpaths)

In [278]:
sfc={}
fc={}
sc={}
num_parcels = nib.load(sfc_netpaths['LOBE'][0]).get_fdata().shape[1]

for dataset in datasets:
    sfc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sfc_netpaths[dataset]] ,axis=0)  
    #for path in sfc_netpaths[dataset]] ,axis=0)
    fc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in fc_netpaths[dataset]] ,axis=0)
    sc[dataset] = np.concatenate(  [ nib.load(path).get_fdata().reshape((1,num_parcels,num_parcels))  for path in sc_netpaths[dataset]] ,axis=0)

## SFC ##

In [279]:
sfc['LOBE'].shape

(23, 7, 7)

### Average per Network - group difference ###

In [280]:
#set diagonal to be nan
x1 = sfc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sfc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

In [281]:
x1.shape

(341, 7, 7)

In [282]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [283]:
x1.shape

(341, 7)

In [284]:
x1

array([[ 0.13693915,  0.05003615,  0.08362471, ...,  0.09395217,
         0.07517542,  0.0838999 ],
       [ 0.04852998,  0.02486698,  0.10213841, ...,  0.0310175 ,
         0.07976645,  0.12653574],
       [ 0.14869973,  0.02676821,  0.1551025 , ...,  0.11501908,
         0.12678167,  0.12783747],
       ...,
       [-0.02605158,  0.01312716,  0.11911449, ...,  0.03174289,
         0.11662567,  0.08154564],
       [ 0.04825305,  0.0414488 ,  0.16903612, ...,  0.09852805,
         0.09137343,  0.14288603],
       [ 0.10234147,  0.06607296,  0.14907431, ...,  0.08672802,
         0.08164697,  0.13172544]])

In [285]:
x2.shape

(23, 7)

In [286]:
#add SFC network avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('NSFC_'+df_network_names))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = list(df_tabular_['participant_id'])
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [287]:
print(df_tabular)

    participant_id dataset        age sex  age_onset  ep_duration  pnes  \
0   sub-HCD0001305     HCP  11.916667   M        NaN          NaN   NaN   
1   sub-HCD0021614     HCP   9.166667   F        NaN          NaN   NaN   
2   sub-HCD0026119     HCP  15.166667   F        NaN          NaN   NaN   
3   sub-HCD0041822     HCP  17.416667   M        NaN          NaN   NaN   
4   sub-HCD0042420     HCP  18.333333   M        NaN          NaN   NaN   
..             ...     ...        ...  ..        ...          ...   ...   
18         sub-029    LOBE  16.000000   M       15.0          1.0   0.0   
19         sub-031    LOBE   9.000000   M        4.0          5.0   0.0   
20         sub-033    LOBE   7.000000   M        4.0          3.0   0.0   
21         sub-035    LOBE   7.000000   M        2.0          5.0   0.0   
22         sub-038    LOBE  10.000000   F        7.0          3.0   0.0   

   focal_seizure_type  generalized_seizures  dre  ...  \
0                 NaN                   Na

### T-test and FDR ###

In [288]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape
pvals

array([0.2363475 , 0.06810607, 0.15582235, 0.80812923, 0.8984041 ,
       0.0316388 , 0.97137546])

In [289]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [290]:
rejected

array([False, False, False, False, False, False, False])

## Functional Connectivity ##

In [291]:
#set diagonal to be nan
x1 = fc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = fc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

### Average FC per network - group difference ###

In [292]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [293]:
x2.shape

(23, 7)

In [294]:
#add FC network avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('NFC_'+df_network_names))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = list(df_tabular_['participant_id'])
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [295]:
print(df_tabular)

    participant_id dataset        age sex  age_onset  ep_duration  pnes  \
0   sub-HCD0001305     HCP  11.916667   M        NaN          NaN   NaN   
1   sub-HCD0021614     HCP   9.166667   F        NaN          NaN   NaN   
2   sub-HCD0026119     HCP  15.166667   F        NaN          NaN   NaN   
3   sub-HCD0041822     HCP  17.416667   M        NaN          NaN   NaN   
4   sub-HCD0042420     HCP  18.333333   M        NaN          NaN   NaN   
..             ...     ...        ...  ..        ...          ...   ...   
18         sub-029    LOBE  16.000000   M       15.0          1.0   0.0   
19         sub-031    LOBE   9.000000   M        4.0          5.0   0.0   
20         sub-033    LOBE   7.000000   M        4.0          3.0   0.0   
21         sub-035    LOBE   7.000000   M        2.0          5.0   0.0   
22         sub-038    LOBE  10.000000   F        7.0          3.0   0.0   

   focal_seizure_type  generalized_seizures  dre  ...  NSFC_Limbic  NSFC_Cont  \
0                 

### T-test & FDR ###

In [296]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape
pvals

array([0.86301361, 0.2210968 , 0.51254288, 0.14047335, 0.59395843,
       0.59135277, 0.28849313])

In [297]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [298]:
rejected

array([False, False, False, False, False, False, False])

## Structural Connectivity ##

In [299]:
#set diagonal to be nan
x1 = sc['HCP']
for s in range(x1.shape[0]):
    conn = x1[s,:,:]    
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x1[s,:,:] = conn

x2 = sc['LOBE']
for s in range(x2.shape[0]):
    conn = x2[s,:,:]
    mask = np.eye(num_parcels)
    conn[mask==1] = np.nan
    x2[s,:,:] = conn

### Average SC per network - group differences ###

In [300]:
x1 = np.nanmean(x1,axis=2)
x2 = np.nanmean(x2,axis=2)

In [301]:
x2.shape

(23, 7)

In [302]:
#add SC network avgs to tabular df

data_={'HCP':x1,'LOBE':x2}

df_list=[]
for dataset in datasets:

    
    df_=pd.DataFrame(data_[dataset],
                 columns=('NSC_'+df_network_names))
    
    df_tabular_ = df_tabular.query(f"dataset == '{dataset}'")
    if dataset == 'HCP':
        df_['participant_id'] = hacky_hcp_subjects
    else:
        df_['participant_id'] = list(df_tabular_['participant_id'])
    
    
    df_merged_ = pd.merge(df_tabular_,df_,how='left',on='participant_id')
    df_list.append(df_merged_)

df_tabular = pd.concat(df_list)

In [303]:
print(df_tabular)

    participant_id dataset        age sex  age_onset  ep_duration  pnes  \
0   sub-HCD0001305     HCP  11.916667   M        NaN          NaN   NaN   
1   sub-HCD0021614     HCP   9.166667   F        NaN          NaN   NaN   
2   sub-HCD0026119     HCP  15.166667   F        NaN          NaN   NaN   
3   sub-HCD0041822     HCP  17.416667   M        NaN          NaN   NaN   
4   sub-HCD0042420     HCP  18.333333   M        NaN          NaN   NaN   
..             ...     ...        ...  ..        ...          ...   ...   
18         sub-029    LOBE  16.000000   M       15.0          1.0   0.0   
19         sub-031    LOBE   9.000000   M        4.0          5.0   0.0   
20         sub-033    LOBE   7.000000   M        4.0          3.0   0.0   
21         sub-035    LOBE   7.000000   M        2.0          5.0   0.0   
22         sub-038    LOBE  10.000000   F        7.0          3.0   0.0   

   focal_seizure_type  generalized_seizures  dre  ...  NFC_Limbic  NFC_Cont  \
0                 Na

### T-test & FDR ###

In [305]:
tstats,pvals,dfs = ttest_ind(x1, x2, alternative='two-sided', usevar='pooled', weights=(None, None), value=0)
tstats.shape
pvals

array([0.26405605, 0.05325768, 0.11895523, 0.08753572, 0.23204956,
       0.78405063, 0.57810951])

In [306]:
rejected,corr_pvals = fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False)

In [307]:
rejected

array([False, False, False, False, False, False, False])

### Save regional and network SFC, SC & FC to masterlist ###

In [304]:
#df_tabular.to_csv('results/analysis/df_tabular.csv',sep='\t',index=False)


df_tabular 

,participant_id,dataset,age,sex,age_onset,ep_duration,pnes,focal_seizure_type,generalized_seizures,dre,...,NFC_Limbic,NFC_Cont,NFC_Default,NSC_Vis,NSC_SomMot,NSC_DorsAttn,NSC_SalVentAttn,NSC_Limbic,NSC_Cont,NSC_Default
0,sub-HCD0001305,HCP,11.916667,M,NaN,NaN,NaN,NaN,NaN,NaN,...,0.200019,0.257366,0.135539,3.885010,5.062987,8.027959,7.580041,5.651460,7.296828,7.935002
1,sub-HCD0021614,HCP,9.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.000347,0.147511,0.039930,4.216869,4.518716,7.629381,7.669835,6.542762,7.380725,8.132475
2,sub-HCD0026119,HCP,15.166667,F,NaN,NaN,NaN,NaN,NaN,NaN,...,0.093052,0.082952,0.052142,4.770188,4.651514,8.898358,7.785734,6.827971,7.798882,8.109647
3,sub-HCD0041822,HCP,17.416667,M,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.001980,-0.008243,0.010302,4.030981,4.773803,8.804379,7.742067,6.960907,8.298339,8.540095
4,sub-HCD0042420,HCP,18.333333,M,NaN,NaN,NaN,NaN,NaN,NaN,...,0.091706,0.153798,0.082643,4.350406,4.851836,9.014367,7.819415,6.471102,8.094806,8.193476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,sub-029,LOBE,16.000000,M,15.0,1.0,0.0,motor,1.0,0.0,...,0.467955,0.392260,0.372756,4.093281,4.358512,7.515517,7.141540,6.457969,6.372687,7.238880
19,sub-031,LOBE,9.000000,M,4.0,5.0,0.0,motor,0.0,NaN,...,0.228163,0.205326,0.162994,4.788529,5.042807,8.379897,7.876680,7.567836,8.409854,8.806705
20,sub-033,LOBE,7.000000,M,4.0,3.0,0.0,non-motor,1.0,1.0,...,-0.029804,0.006911,-0.027776,4.030132,4.791794,7.306059,7.469537,6.275113,7.390206,8.429542
21,sub-035,LOBE,7.000000,M,2.0,5.0,0.0,motor,1.0,0.0,...,0.035206,-0.010172,0.008450,4.027565,4.504583,7.768704,7.925532,6.047853,7.735365,8.119764


# Taking clinical phenotypes into account #

## Temporal EEG Localization vs Non-Temporal ##

In [308]:
df_clinical = pd.read_table('clinical/participants.tsv')

temporal = df_clinical[df_clinical["loc_eeg_temporal"] == 1]
nontemporal = df_clinical[df_clinical["loc_eeg_temporal"] != 1]

df_temporal = temporal["participant_id"]
df_nontemporal = nontemporal["participant_id"]

In [309]:
print(df_nontemporal)

0     sub-006
3     sub-010
8     sub-017
10    sub-020
11    sub-021
12    sub-022
13    sub-024
16    sub-027
17    sub-029
18    sub-031
19    sub-033
21    sub-038
Name: participant_id, dtype: object
